In [48]:
!pip install deep-translator

In [49]:
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
from deep_translator import GoogleTranslator

In [50]:
nltk.download('vader_lexicon')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/duarte/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/duarte/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /home/duarte/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/duarte/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [51]:
# Step 2: Load the dataset
file_path = "results-publico.csv"
columns = ["term", "url", "text", "title"]
df = pd.read_csv(file_path, header=1,  names=columns)

In [55]:
# Check for missing values; address any missing data if necessary
has_missing = df.isnull().values.any()
if has_missing:
    print('Missing values found. Dropping rows with missing values.')
    df.dropna(inplace=True)
else:
    print('No missing values found.')
df['text'] = df['text'].str.strip().str.lower()
df.drop_duplicates(subset='text', keep='first', inplace=True)

Missing values found. Dropping rows with missing values.


In [61]:
translator = GoogleTranslator(source='pt', target='en')
global i 
i = 0

def translate_text(sentences):
    global i
    i += 1
    print(f"#{i}: Translating {len(sentences)} sentences")
    return translator.translate_batch(sentences)

def spli_in_5000_sentences(text):
    return [text[i:i+4998] for i in range(0, len(text), 4998)]
    

# Step 4: Translate the text
df['sentences'] = df['text'].apply(spli_in_5000_sentences)
df['translated_text'] = df['sentences'].apply(translate_text)
df.head()

#1: Translating 2 sentences
#2: Translating 1 sentences
#3: Translating 1 sentences
#4: Translating 1 sentences
#5: Translating 1 sentences
#6: Translating 3 sentences
#7: Translating 21 sentences
#8: Translating 21 sentences
#9: Translating 1 sentences
#10: Translating 1 sentences
#11: Translating 1 sentences
#12: Translating 1 sentences
#13: Translating 1 sentences
#14: Translating 1 sentences
#15: Translating 1 sentences
#16: Translating 2 sentences
#17: Translating 2 sentences
#18: Translating 1 sentences
#19: Translating 1 sentences
#20: Translating 1 sentences
#21: Translating 1 sentences
#22: Translating 2 sentences
#23: Translating 2 sentences
#24: Translating 1 sentences
#25: Translating 6 sentences
#26: Translating 1 sentences
#27: Translating 5 sentences
#28: Translating 21 sentences
#29: Translating 21 sentences
#30: Translating 21 sentences
#31: Translating 21 sentences
#32: Translating 1 sentences
#33: Translating 1 sentences
#34: Translating 1 sentences
#35: Translating 

KeyboardInterrupt: 

In [ ]:
df['translated_text'] = df['translated_text'].apply(lambda x: ' '.join(x))
df.head()

# remove setences column
df = df.drop(columns=['sentences'])
df.head()

df.to_csv("results-publico-translated.csv", index=False)

,term,url,text,sentences,translated_text
0,ABEL BAPTISTA,https://arquivo.pt/noFrame/replay/200803111828...,"De acordo com Miguel Relvas, Guilhermino Rodri...","[De acordo com Miguel Relvas, Guilhermino Rodr...","According to Miguel Relvas, Guilhermino Rodrig..."
1,ABEL BAPTISTA,https://arquivo.pt/noFrame/replay/200506211901...,Composição do III Governo Constitucional Toma...,[Composição do III Governo Constitucional Tom...,Composition of the III Constitutional Governme...
2,ABEL BAPTISTA,https://arquivo.pt/noFrame/replay/200506211827...,CORRESPONDENTES NACIONAIS Alcobaça: Amadeu Lea...,[CORRESPONDENTES NACIONAIS Alcobaça: Amadeu Le...,NATIONAL CORRESPONDENTS Sines: Raúl Oliveira; ...
3,ABEL BAPTISTA,https://arquivo.pt/noFrame/replay/200602160601...,15 de Fevereiro 2006 Fátima Felgueiras recebeu...,[15 de Fevereiro 2006 Fátima Felgueiras recebe...,15 February 2006 Fátima Felgueiras received a ...
4,ABEL BAPTISTA,https://arquivo.pt/noFrame/replay/200602270803...,25 de Fevereiro 2006 Escutas provocam debate a...,[25 de Fevereiro 2006 Escutas provocam debate ...,25 February 2006 Wiretaps spark heated debate ...


In [45]:
lemmatizer = WordNetLemmatizer()

stop_words = set(stopwords.words('english'))

stop_words.add('goes')
stop_words.add('newspaper')
stop_words.add('journalist')
stop_words.add('journalists')
stop_words.add('pt')
stop_words.add('rsf')
stop_words.add('aacs')
stop_words.add('still')
stop_words.add('year')
stop_words.add('today')
stop_words.add('day')
stop_words.add('to have')
stop_words.add('about')
stop_words.add('hurts')
stop_words.add('yesterday')
stop_words.add('where')
stop_words.add('one')
stop_words.add('two')
stop_words.add('three')
stop_words.add('four')
stop_words.add('five')
stop_words.add('six')
stop_words.add('seven')
stop_words.add('eight')
stop_words.add('nine')
stop_words.add('ten')
stop_words.add('because')
stop_words.add('years')
stop_words.add('said')

# Tokenize the text
def tokenize(text):
    tokens = word_tokenize(text, language='english')
    for token in tokens:
        lword = token.lower()
        if lword.isalpha() and (lword not in stop_words) and (len(lword) > 2):
            lemWord = lemmatizer.lemmatize(lword)
            yield lemWord 
            

# Step 3: Tokenize the text
df['tokens'] = df['translated_text'].apply(lambda x: [t for t in tokenize(x)])
# join the tokens back into a single string
df['cleaned_text'] = df['tokens'].apply(lambda x: ' '.join(x))
df.head()

,term,url,text,sentences,translated_text,compound_score,sentiment,tokens,cleaned_text
0,ABEL BAPTISTA,https://arquivo.pt/noFrame/replay/200803111828...,"De acordo com Miguel Relvas, Guilhermino Rodri...","[De acordo com Miguel Relvas, Guilhermino Rodr...","According to Miguel Relvas, Guilhermino Rodrig...",0.9881,positive,"[according, miguel, relvas, guilhermino, rodri...",according miguel relvas guilhermino rodrigues ...
1,ABEL BAPTISTA,https://arquivo.pt/noFrame/replay/200506211901...,Composição do III Governo Constitucional Toma...,[Composição do III Governo Constitucional Tom...,Composition of the III Constitutional Governme...,-0.9991,negative,"[composition, iii, constitutional, government,...",composition iii constitutional government inau...
2,ABEL BAPTISTA,https://arquivo.pt/noFrame/replay/200506211827...,CORRESPONDENTES NACIONAIS Alcobaça: Amadeu Lea...,[CORRESPONDENTES NACIONAIS Alcobaça: Amadeu Le...,NATIONAL CORRESPONDENTS Sines: Raúl Oliveira; ...,0.0000,negative,"[national, correspondent, sine, raúl, oliveira...",national correspondent sine raúl oliveira setú...
3,ABEL BAPTISTA,https://arquivo.pt/noFrame/replay/200602160601...,15 de Fevereiro 2006 Fátima Felgueiras recebeu...,[15 de Fevereiro 2006 Fátima Felgueiras recebe...,15 February 2006 Fátima Felgueiras received a ...,-0.9995,negative,"[february, fátima, felgueiras, received, month...",february fátima felgueiras received monthly pe...
4,ABEL BAPTISTA,https://arquivo.pt/noFrame/replay/200602270803...,25 de Fevereiro 2006 Escutas provocam debate a...,[25 de Fevereiro 2006 Escutas provocam debate ...,25 February 2006 Wiretaps spark heated debate ...,-0.9996,negative,"[february, wiretap, spark, heated, debate, amo...",february wiretap spark heated debate among law...


In [47]:
# Step 3: Initialize VADER
analyzer = SentimentIntensityAnalyzer()

# Step 4: Analyze Sentiment
def analyze_sentiment(text):
    scores = analyzer.polarity_scores(text)
    return scores['compound']

df['compound_score'] = df['cleaned_text'].apply(analyze_sentiment)

# Step 5: Map Compound Scores to Sentiment Labels
def map_sentiment(compound):
    return "positive" if compound > 0 else "negative"

df['sentiment'] = df['compound_score'].apply(map_sentiment)

# Step 6: Save results or preview
print(df[['text', 'compound_score', 'sentiment']].head())
#df.to_csv("Twitter_Sentiment_Analysis_with_VADER.csv", index=False)

                                                text  compound_score sentiment
0  De acordo com Miguel Relvas, Guilhermino Rodri...          0.9912  positive
1  Composição do III Governo Constitucional  Toma...          0.9201  positive
2  CORRESPONDENTES NACIONAIS Alcobaça: Amadeu Lea...          0.0000  negative
3  15 de Fevereiro 2006 Fátima Felgueiras recebeu...         -0.9982  negative
4  25 de Fevereiro 2006 Escutas provocam debate a...         -0.9973  negative
